In [55]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime

convertTime = lambda x : datetime.strptime(x, "%Y/%m/%d - %H:%M:%S").strftime("%Y-%m-%dT%H:%M:%SZ")

In [56]:
acquisitionGPS = pd.read_csv('results/acquisitionsGPS/gps1.csv')
acquisitionTEL = pd.read_csv('results/acquisitionsTELEPHONE/CSV/acquisition1.csv')

In [57]:
acquisitionGPS['UTC'] = acquisitionGPS['UTC'].apply(convertTime)
acquisitionGPS['UTC'] = pd.to_datetime(acquisitionGPS['UTC'])
acquisitionTEL['UTC'] = pd.to_datetime(acquisitionTEL['UTC'])

In [58]:
merged_df = pd.merge_asof(acquisitionGPS.sort_values('UTC'), acquisitionTEL.sort_values('UTC'), on='UTC', suffixes=('_1', '_2'))
merged_df

,Timestamp,UTC,Latitude_1,Longitude_1,Latitude_2,Longitude_2
0,1.741710e+09,2025-03-11 16:17:32+00:00,49.384318,1.068916,NaN,NaN
1,1.741710e+09,2025-03-11 16:17:33+00:00,49.384318,1.068916,NaN,NaN
2,1.741710e+09,2025-03-11 16:17:34+00:00,49.384318,1.068915,NaN,NaN
3,1.741710e+09,2025-03-11 16:17:56+00:00,49.384316,1.068903,NaN,NaN
4,1.741710e+09,2025-03-11 16:17:57+00:00,49.384316,1.068903,NaN,NaN
...,...,...,...,...,...,...
493,1.741710e+09,2025-03-11 16:26:53+00:00,49.384899,1.068054,49.384380,1.068232
494,1.741710e+09,2025-03-11 16:26:54+00:00,49.384909,1.068036,49.384370,1.068218
495,1.741710e+09,2025-03-11 16:26:55+00:00,49.384922,1.068017,49.384378,1.068212
496,1.741710e+09,2025-03-11 16:26:56+00:00,49.384934,1.068000,49.384358,1.068197


In [59]:
from geopy.distance import geodesic

def haversine_distance(lat1, lon1, lat2, lon2):
    return geodesic((lat1, lon1), (lat2, lon2)).meters

# Calcul de la distance entre chaque point des deux jeux de données
merged_df['distance'] = merged_df.apply(lambda row: haversine_distance(
    row['Latitude_1'], row['Longitude_1'], row['Latitude_2'], row['Longitude_2']
), axis=1)

print(merged_df[['UTC', 'distance']].head())  # Voir les écarts


ValueError: Point coordinates must be finite. (nan, nan, 0.0) has been passed as coordinates.